# Analyzing images
Comparing networks of dense layers to convolutional networks on simple image analyses.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

2023-04-04 12:17:45.793713: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-04 12:17:46.223691: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-04 12:17:46.225938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 12:17:47.627246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Generator functions for sample images
Defining functions that generate images with circles/squares in it. See docstring for details

In [137]:
def create_image_w_square(im_size_x=200, im_size_y=200, fill_color=[255, 255, 255],
                          sq_size=30, sq_color=[255, 128, 0]):
    '''Creates a numpy array with three dimensions, which represents an color image.
    The picture will hold a square on an arbitary position.
    Size of image and square, as well as the color of background and square my be specified.
    Besides the image the position of the square (its center position) and its area in pixels is returned'''
    # Generating image with background color
    image = np.full(shape=(im_size_x, im_size_y, 3), fill_value=fill_color)
    # Determing random position for square
    coord_x = np.random.randint(low=0, high=(im_size_x - sq_size))
    coord_y = np.random.randint(low=0, high=(im_size_y - sq_size))
    # Filling square in the image by scpecifying its color value on all three layers
    for i in range(3):
        image[coord_x:coord_x + sq_size, coord_y:coord_y + sq_size, i] = sq_color[i]
    return image, (coord_x + sq_size/2, coord_y + sq_size/2), sq_size**2

def create_image_w_circle(im_size_x=200, im_size_y=200, fill_color=[255, 255, 255],
                          circle_r=17, circle_color=[255, 128, 0]):
    '''Creates a numpy array with three dimensions, which represents an color image.
    The picture will hold a circle on an arbitary position.
    Size of image and circle, as well as the color of background and circle my be specified.
    Besides the image the position of the circle and its area in pixels is returned'''
    # Generating image with background color
    image = np.full(shape=(im_size_x, im_size_y, 3), fill_value=fill_color)
    # Determing coordinates for center of circle
    coord_x = np.random.randint(low=circle_r, high=(im_size_x - circle_r))
    coord_y = np.random.randint(low=circle_r, high=(im_size_y - circle_r))
    # Filling circle in the image by specifying its color value on all three layers
    for i in range(3):
        # Creating a meshgrid of image size and than calculate from that, whether position is in circle
        mesh = np.meshgrid(range(im_size_y), range(im_size_x))
        mesh[0] -= coord_y
        mesh[1] -= coord_x
        mask = np.sqrt(mesh[0]**2 + mesh[1]**2) < circle_r
        # Write new color on image, where pixels are within radius of circle
        image[mask, i] = circle_color[i]
    return image, (coord_x, coord_y), np.pi*circle_r**2

## Generating sample image 1
200 x 200 pixel images, containing same sized squares and circles.

In [138]:
# Defining number of samples
sample_size = 2000

# Filling list with sample images, which contain squares or circles. Holding information about 
# circle or quare in seperate list.
samples = []
target = []
for i in range(sample_size):
    if np.random.randint(low=0, high=2):
        image, *_ = create_image_w_square()
        samples.append(image)
        target.append(1)
    else:
        image, *_ = create_image_w_circle()
        samples.append(image)
        target.append(0)